# Creating New Models

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from materialite import Material
from materialite.models import Model

In Materialite, models are classes that are designed to operate on a `Material` in some way and return a new `Material`. Models have three requirements:
1. Models must be subclasses of the Materialite `Material` class. 
2. The inputs of the model constructor (`__init__`) should be model parameters that are not associated with the `Material` (e.g., time incrementation parameters, applied loads, etc.) and stores them as attributes of the model.
3. Models must have a method called `run`. The first input to the run method must be a `Material`. Typically, we also have keyword arguments for labels of fields in the `Material` that the model will use. The basic workflow in the run method is (1) pull required fields from the `Material`; (2) do the actual calculations associated with the model (this step should not depend on any details of the `Material`); (3) after the model is done running, create a new `Material` with outputs from the model.

Here, we will make a simple model that does a uniaxial decoupled elasticity simulation. The model input is the applied uniaxial strain. The `Material` needs to have a field that provides Young's modulus at each point.

In [ ]:
class UniaxialElasticity(Model):
    def __init__(self, applied_strain):
        self.applied_strain = applied_strain

    def run(self, material, modulus_field="modulus"):
        # inputs from Material
        moduli = material.extract(modulus_field)
        # do calculations
        stress = moduli * self.applied_strain
        # output to Material
        return material.create_fields({"stress": stress})

Create an instance of the model

In [ ]:
applied_strain = 0.001
model = UniaxialElasticity(applied_strain)

Create a `Material` with the required field.

In [ ]:
material = Material()
rng = np.random.default_rng(12345)
moduli = rng.normal(1000, 10, size=(material.num_points,))
material = material.create_fields({"modulus": moduli})

There are three ways to run a model with a `Material`.

1. Call `model.run(material)`

In [ ]:
material1 = model.run(material)

2. Call `model(material)`

In [ ]:
material2 = model(material)

3. Call `material.run(model)`. This is useful because it allows method chaining in a `Material`. For example, we could both add the modulus field and run the model like this:

In [ ]:
material3 = Material().create_fields({"modulus": moduli}).run(model)

In [ ]:
print(np.allclose(material1.extract("stress"), material2.extract("stress")))
print(np.allclose(material1.extract("stress"), material3.extract("stress")))

Some notes:
* There is significant flexibility in how the `run` method actually runs the model. The only requirements are that the first input to the method is a `Material`, and that the method returns a `Material`. The mechanics of actually running the model are up to you. Depending on the complexity of the model, it might make sense to split off some of the calculations to other methods or functions that are called within run.
* If the model takes many time steps (like most physics-based models will do), you may want to store outputs at multiple steps. For now, we are using `material.state` for this purpose. `state` is a dictionary that is an attribute of any `Material`. The model can add any key-value pairs that you would like to this dictionary. `material.state` is also useful for storing model outputs that don't make sense as pointwise fields (e.g., some average measure, the total energy in a Potts model, etc.) This is probably not the long-term solution, but it works for now.